In [15]:
import pandas as pd  
pd.options.mode.chained_assignment = None 
import os, json, time,calendar, re  

In [139]:
dirs = [f.path.replace('\\','/').replace('Data/','') for f in os.scandir('Data') if f.is_dir()]
with open('Static/js/datadirs.js', 'w') as f:
    f.write(f'var datadirs = {str( dirs )};')


In [164]:
def catReplace(c):
    c=str(c)
    if re.search('NUTRITION|TEA|SAUCE|SUGAR|DRINK|HERB|POPCORN|FLOUR|BREAD|SOUP|HONEY|CEREAL',c):
        return 'GROCERY'
    if re.search('PANTS|DRESS|SHIRT|BOOT|SHOES|HAT|SOCKS|WATCH|PAJAMAS|APPAREL|SWEATER',c):
        return 'CLOTHES' 
    if re.search('SHAMPOO|HAIR|CONDITIONER|HEALTH_|EYELID_FLOSS|MEDICA',c): 
        return 'PERSONAL'  
    if re.search('SKIN',c): 
        return 'SKIN'              
    if re.search('MONITOR|POWER_BANK|BOOK|MOUSE_|_PEN|_MOUSE|HEADPHONE|_CABLE|BATTERY|FILE_|REMOTE_|AUTO_|POWER_',c): 
        return 'OFFICE'    
    if re.search('OVEN|KITCHEN|_BAG|COOKER|COOKWARE|FOOD_|FRYER|WEIGH_SCALE',c): 
        return 'KITCHEN'  
    if re.search('_HELMET|_PUMP|OUTDOOR_RECREATION_PRODUCT|BICYCLE_',c): 
        return 'BIKE' 
    if re.search('BATH|CLEANING_|UTDOOR_|_ADAPTER|STORAGE|BED_',c): 
        return 'HOME'  
    if re.search('CRAFT_|PHONE|SCREEN_P|PET_|ANIMAL|LINT|_DEVICE|TOY|FABRIC|YARN',c): 
        return 'MISC'                                              
    return c

In [168]:
def processDataDirectory(dir):
    files = os.listdir(f'Data/{dir}') 
    df = pd.DataFrame()
    for f in files:
        df1 = pd.read_csv(f'Data/{dir}/{f}')
        df = pd.concat([df1, df], axis=0, ignore_index=True)   
    df = df.rename(columns ={
                          'Order Date':'TransactionDate'
                         ,'Transaction Date':'TransactionDate'
                         ,'Item Total':'Amount'
                         ,'Title':'Description'})  
    df=df.loc[:, ['TransactionDate', 'Description', 'Category', 'Amount' ] ]
    df=df.drop_duplicates(subset=['TransactionDate','Description','Amount'], keep="first")
    df['Description']=df['Description'].fillna('')
    df['Amount'] = df['Amount'].apply(lambda x: str(x).replace('$',''))
    df['Amount'] = df['Amount'].astype('float').abs()    
    df['Month'] = df['TransactionDate'].apply(lambda x: x.split('/')[0])  
    df['Year'] = df['TransactionDate'].apply(lambda x: x.split('/')[2])   
    df['Category'] = df['Category'].apply(lambda c: catReplace(c) ) 
    
    df = df.merge(
        df.groupby('Category', as_index=False).sum()
        , left_on='Category', right_on='Category', suffixes=('', 'CatSum'))

    df = df.merge(
        df.groupby(['Month', 'Year'], as_index=False).agg({'Amount': 'sum'}) 
        , left_on=['Month', 'Year'], right_on=['Month', 'Year'], suffixes=('', 'MonthSum'))
  
    #print( df )

    df = df.reindex(columns= ['TransactionDate', 'Year', 'Month', 'Description', 'Category','Amount','AmountCatSum','AmountMonthSum']) 
    df['AmountCatSum'] = df['AmountCatSum'].astype('float')
    df['AmountMonthSum'] = df['AmountMonthSum'].astype('float')
    df.to_csv(f'Data/df_{dir}.csv', index=False) 
    df.to_json(f'Data/df_{dir}.json', orient='records')
    return df

[processDataDirectory(d) for d in dirs]



[    TransactionDate Year Month  \
 0          01/03/20   20    01   
 1          01/12/20   20    01   
 2          01/28/20   20    01   
 3          01/09/20   20    01   
 4          01/09/20   20    01   
 ..              ...  ...   ...   
 257        01/29/21   21    01   
 258        01/29/21   21    01   
 259        01/29/21   21    01   
 260        01/29/21   21    01   
 261        01/05/21   21    01   
 
                                            Description        Category  \
 0    Kernel Season's Popcorn Seasoning, Sour Cream ...         GROCERY   
 1    Probiotics 60 Billion CFU - Dr. Approved Probi...         GROCERY   
 2    Davidson's Tea Bulk, Herb Chamomile Flower, 16...         GROCERY   
 3    wirarpa Women's High Waisted Cotton Underwear ...         CLOTHES   
 4    Amazon Brand - Lark & Ro Women's Wide Leg Pont...         CLOTHES   
 ..                                                 ...             ...   
 257  Brooklyn Botany Fractionated Coconut Oil for S.

In [17]:
files = os.listdir('Data/ccard') 
df = pd.DataFrame()
for f in files:
    df1 = pd.read_csv(f'Data/ccard/{f}')
    df = pd.concat([df1, df], axis=0, ignore_index=True)   
df = df.rename(columns ={'Transaction Date':'TransactionDate'}) 
df=df.loc[:, ['TransactionDate', 'Description', 'Category', 'Amount' ] ]
df[0:2]

,TransactionDate,Description,Category,Amount
0,1/19/2021,Zipcar Toll JAN 07,Travel,-0.85
1,1/19/2021,Zipcar Toll JAN 07,Travel,-0.85


In [18]:
df_ccard = df.drop_duplicates(subset=['TransactionDate','Description','Amount'], keep="first")
df_ccard.head(2)

,TransactionDate,Description,Category,Amount
0,1/19/2021,Zipcar Toll JAN 07,Travel,-0.85
3,1/18/2021,MOMIJI RESTAURANT.,Food & Drink,-112.36


## Save Clean to new CSV

In [19]:
df = df_ccard
df = df.loc[df['Amount'] < 0] 
df['Amount'] = df['Amount'].abs()
# Set new column Day = the day extracted from Tranaction Date column
df['Day'] = df['TransactionDate'].apply(lambda x: x.split('/')[1]) 
# Set new column Month = the month extracted from Tranaction Date column
df['Month'] = df['TransactionDate'].apply(lambda x: x.split('/')[0]) 
# Set new column Year = the month extracted from Tranaction Date column
df['Year'] = df['TransactionDate'].apply(lambda x: x.split('/')[2]) 
# Reorder Columns
df = df.reindex(columns= ['TransactionDate', 'Year', 'Month', 'Day', 'Description', 'Category','Amount'])   
df_ccard = df 

In [20]:
df = df_ccard
#save a clean csv && json
df.to_csv('Data/df_ccard.csv', index=False) 
df.to_json('Data/df_ccard.json', orient='records')

In [21]:
files = os.listdir('Data/amazon') 
df = pd.DataFrame()
for f in files:
    df1 = pd.read_csv(f'Data/amazon/{f}')
    df = pd.concat([df1, df], axis=0, ignore_index=True)   
df = df.rename(columns ={'Order Date':'TransactionDate'
                         ,'Trans':'TransactionDate'
                         ,'Item Total':'Amount'
                         ,'Title':'Description'
                        }) 
df=df.loc[:, ['TransactionDate', 'Description', 'Category', 'Amount' ] ]
df[0:2]

,TransactionDate,Description,Category,Amount
0,01/03/20,"Kernel Season's Popcorn Seasoning, Sour Cream ...",GROCERY,$11.88
1,01/09/20,wirarpa Women's High Waisted Cotton Underwear ...,UNDERPANTS,$24.37


In [22]:
df_amazon = df.drop_duplicates(subset=['TransactionDate','Description','Amount'], keep="first")
df_amazon.head(2)

,TransactionDate,Description,Category,Amount
0,01/03/20,"Kernel Season's Popcorn Seasoning, Sour Cream ...",GROCERY,$11.88
1,01/09/20,wirarpa Women's High Waisted Cotton Underwear ...,UNDERPANTS,$24.37


In [23]:
df=df_amazon
df['Description']=df['Description'].fillna('')
df['Amount'] = df['Amount'].apply(lambda x: x.replace('$','')).astype('float').abs() 
df['Day'] = df['TransactionDate'].apply(lambda x: x.split('/')[1])  
df['Month'] = df['TransactionDate'].apply(lambda x: x.split('/')[0])  
df['Year'] = df['TransactionDate'].apply(lambda x: x.split('/')[2])  
df = df.reindex(columns= ['TransactionDate', 'Year', 'Month', 'Day', 'Description', 'Category','Amount'])   
df

,TransactionDate,Year,Month,Day,Description,Category,Amount
0,01/03/20,20,01,03,"Kernel Season's Popcorn Seasoning, Sour Cream ...",GROCERY,11.88
1,01/09/20,20,01,09,wirarpa Women's High Waisted Cotton Underwear ...,UNDERPANTS,24.37
2,01/09/20,20,01,09,"Neutrogena Nourishing Eyeliner Pencil, Built-i...",EYELID_COLOR,7.92
3,01/09/20,20,01,09,Amazon Brand - Lark & Ro Women's Wide Leg Pont...,PANTS,37.10
4,01/09/20,20,01,09,Women's Elegant Vintage Cotton Casual Floral P...,DRESS,28.61
...,...,...,...,...,...,...,...
258,01/22/21,21,01,22,"EXPO 86661 Low-Odor Dry Erase Markers, Fine Po...",MARKING_PEN,4.73
259,01/29/21,21,01,29,Brooklyn Botany Fractionated Coconut Oil for S...,SKIN_MOISTURIZER,14.79
260,01/29/21,21,01,29,Brooklyn Botany Castor Oil – 100% Pure and Col...,SKIN_MOISTURIZER,11.29
261,01/29/21,21,01,29,Solimo Moisturizing Coconut Milk and Jasmine N...,SKIN_CLEANING_AGENT,17.92


In [10]:
df.to_csv('Data/df_amazon.csv', index=False) 
df.to_json('Data/df_amazon.json', orient='records')

In [11]:
df.loc[df.Category.str.contains('Headphones', case=False, regex=False, na=False)] 

,TransactionDate,Year,Month,Day,Description,Category,Amount
41,03/12/20,20,03,12,Paula's Choice Skin Perfecting 2% BHA Liquid S...,HEADPHONES,38.69
151,08/19/20,20,08,19,"CUKOR Hot Plate,Electric Single Burner for Coo...",HEADPHONES,35.13
197,10/21/20,20,10,21,"Earbuds Wired with Microphone Pack of 5, Noise...",HEADPHONES,12.71
198,10/21/20,20,10,21,"Earbuds Headphones with Microphone 5 Pack, Ear...",HEADPHONES,12.71
